In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [86]:
data = pd.read_csv('lesser_model_data.csv')

In [87]:
print(data.columns)

Index(['Region', 'Region ID', 'MarCases', 'AprCases', 'MayCases', 'JunCases',
       'MarHome', 'AprHome', 'MayHome', 'JunHome', 'MarTest', 'AprTest',
       'MayTest', 'JunTest'],
      dtype='object')


In [88]:
def get_probabilities(states):
    num_total = np.shape(states)[0] * np.shape(states)[1]
    num_ones = np.shape(np.where(states == 1))[1]
    num_zeros = num_total - num_ones

    prob_one = num_ones/num_total
    prob_zero = num_zeros/num_total
    
    assert round(prob_one+prob_zero, 3) == 1. 
    
    return prob_one, prob_zero


statedataStayHome = {'MarHome' : data['MarHome'], 'AprHome' : data['AprHome'], 'MayHome' : data['MayHome'], 'JunHome' : data['JunHome']}
statesStayHome = pd.DataFrame(data=statedataStayHome)

statedataTests = {'MarTest' : data['MarTest'], 'AprTest' : data['AprTest'], 'MayTest' : data['MayTest'], 'JunTest' : data['JunTest']}
statesTests = pd.DataFrame(data=statedataTests)

prob_goodTests, prob_badTests = get_probabilities(statesTests)
prob_stayHome, prob_noStayHome = get_probabilities(statesStayHome)

In [91]:
def get_conditional_probability(Astates, Bstates, Cstates):
    """A and B are parent nodes. C is child node"""
    
    num111, num101, num011, num001, num110, num100, num010, num000 = 0, 0, 0, 0, 0, 0, 0, 0
    num11, num01, num10, num00 = 0,0,0,0
    
    #Add up states where C=1
    indC_one_row = np.where(Cstates == 1)[0] 
    indC_one_col = np.where(Cstates == 1)[1] 


    for i in range(len(indC_one_col)): # Number of times C=1
        A = Astates.iloc[indC_one_row[i], indC_one_col[i]]
        B = Bstates.iloc[indC_one_row[i], indC_one_col[i]]
        if A == 1:
            if B == 1:
                num111 += 1
                num11 += 1
            elif B == 0:
                num101 += 1
                num10 += 1
        elif A == 0:
            if B == 1:
                num011 += 1
                num01 += 1
            elif B == 0:
                num001 += 1
                num00 += 1
                
    # Add up states where C=0
    indC_zero_row = np.where(Cstates == 0)[0] 
    indC_zero_col = np.where(Cstates == 0)[1] 


    for i in range(len(indC_zero_col)): # Number of times C=0
        A = Astates.iloc[indC_zero_row[i], indC_zero_col[i]]
        B = Bstates.iloc[indC_zero_row[i], indC_zero_col[i]]
        if A == 1:
            if B == 1:
                num110 += 1
                num11 += 1
            elif B == 0:
                num100 += 1
                num10 += 1
        elif A == 0:
            if B == 1:
                num010 += 1
                num01 += 1
            elif B == 0:
                num000 += 1
                num00 += 1
    
    # Calculate conditional probabilities
    num_total = np.shape(Astates)[0] * np.shape(Astates)[1]
    
    P111 = num111/num11
    P101 = num101/num10
    P011 = num011/num01
    P001 = num001/num00
    P110 = num110/num11
    P100 = num100/num10
    P010 = num010/num01
    P000 = num000/num00
    
    #conditional_probabilities = np.array([['111','101','011','001','110','100','010','000'],[P111,P101,P011,P001,P110,P100,P010,P000]]).T
    states = np.array(['111','101','011','001','110','100','010','000'])
    conditional_probabilities = np.array([P111,P101,P011,P001,P110,P100,P010,P000])
    
    return states, conditional_probabilities
                        

statedataCases = {'MarCases' : data['MarCases'], 'AprCases' : data['AprCases'], 'MayCases' : data['MayCases'], 'JunCases' : data['JunCases']}
statesCases = pd.DataFrame(data=statedataCases)

states, cond_probs = get_conditional_probability(statesStayHome, statesTests, statesCases)
print(states, cond_probs)

['111' '101' '011' '001' '110' '100' '010' '000'] [0.76923077 0.29310345 0.15873016 0.18181818 0.23076923 0.70689655
 0.84126984 0.81818182]
